<div style="color:#00BFFF">

# Nowcasting Consumer Expenditure: 

<div style="color:#00BFFF">

### About the Data

#### Primary Dataset Description

**Short Description:** The primary dataset is "Table 1.1.5. Gross Domestic Product" from the U.S. Bureau of Economic Analysis. It comprises seasonally adjusted quarterly U.S. Gross Domestic Product (GDP) rates in billions of dollars.

**Relevance:** The dataset's detailed information on U.S. GDP over several years is integral to the project's goal of nowcasting consumption. The data's granularity and time-series nature will allow for comprehensive analysis and identification of trends, making it pivotal for the project's success.

**Data frequency:** The data reflecting the economic output of the United States is crucial for analyzing economic trends and growth patterns. The presentation of data is done quarterly by the GDP component.

**Location:** Available at [U.S. Bureau of Economic Analysis](https://apps.bea.gov/iTable/?reqid=19&step=2&isuri=1&categories=survey&_gl=1*j1lvlb*_ga*MTk0MDMyMjk0MC4xNzA1NDk1NTk4*_ga_J4698JNNFT*MTcwNTQ5NTU5OC4xLjEuMTcwNTQ5NzA2MC42MC4wLjA.#eyJhcHBpZCI6MTksInN0ZXBzIjpbMSwyLDMsM10sImRhdGEiOltbImNhdGVnb3JpZXMiLCJTdXJ2ZXkiXSxbIk5JUEFfVGFibGVfTGlzdCIsIjUiXSxbIkZpcnN0X1llYXIiLCIxOTQ3Il0sWyJMYXN0X1llYXIiLCIyMDIzIl0sWyJTY2FsZSIsIi05Il0sWyJTZXJpZXMiLCJRIl1dfQ==). ([BEA](https://apps.bea.gov/iTable/?reqid=19&step=2&isuri=1&categories=survey&_gl=1*j1lvlb*_ga*MTk0MDMyMjk0MC4xNzA1NDk1NTk4*_ga_J4698JNNFT*MTcwNTQ5NTU5OC4xLjEuMTcwNTQ5NzA2MC42MC4wLjA.#eyJhcHBpZCI6MTksInN0ZXBzIjpbMSwyLDMsM10sImRhdGEiOltbImNhdGVnb3JpZXMiLCJTdXJ2ZXkiXSxbIk5JUEFfVGFibGVfTGlzdCIsIjUiXSxbIkZpcnN0X1llYXIiLCIxOTQ3Il0sWyJMYXN0X1llYXIiLCIyMDIzIl0sWyJTY2FsZSIsIi05Il0sWyJTZXJpZXMiLCJRIl1dfQ==))

**Format:** CSV

**Access Method:** The dataset is readily available and can be easily accessed and downloaded directly from the U.S. Bureau of Economic Analysis website.


#### Secondary Datasets

##### Federal Reserve Economic Data (FRED)

**Short Description:** This dataset is sourced from the Federal Reserve Bank of St. Louis's FRED macroeconomic database. It contains a variety of economic data points available at monthly intervals, with a particular focus on US GDP data. The data covers consumer spending indicators, a crucial component of the Gross Domestic Product (GDP).

**Relevance**: Complements the primary dataset with additional economic indicators, useful for cross-referencing and correlation analysis.

**Data frequency:** The monthly frequency of this dataset provides a more detailed temporal resolution than the primary dataset, which may reveal more immediate economic trends. This granularity will be useful in identifying more immediate proxies for nowcasting.

**Estimated Size**: 0.6MB

**Location**: https://research.stlouisfed.org/econ/mccracken/fred-databases/

**Format**: CSV.

**Access Method**: Direct download.

<div style="color:#00BFFF">

### Setup Environment and import libraries

In [ ]:
# Activate the virtual environment by running in terminal: 
# python -m venv myenv
# source myenv/bin/activate
# ! source /myenv/bin/activate

# ------- PIP INSTALLS -------
# ! pip install --upgrade pip
# ! pip install -r requirements.txt

# Run the imports file
%matplotlib inline

In [ ]:
# ------- Standard Library Imports -------
import warnings
from pprint import pprint
# from typing import List

# ------- Third-Party Library Imports -------
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt

# Remove warnings
warnings.filterwarnings('ignore')

# Set the display options
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)  

<div style="color:#00BFFF">

### Load datasets

<div style="color:#00BFFF">

##### Load and preprocesses BEA Dataset


**Loads and preprocesses** the GDP data from a CSV file. Process a DataFrame to create a structured description column.

**Handling Missing Values**: Utilize median imputation for missing values, as it's less influenced by outliers and provides a more representative central tendency.

**Outliers and Anomalies**: Apply Interquartile Range (IQR) or Z-score analysis to identify and address outliers. This step ensures the integrity of data by minimizing the impact of extreme values.

In [ ]:
from utils.bea_load_data import load_and_preprocess_gdp_data,create_structured_description, create_short_description, transform_date_formats

file_path = './data/bea/bea_usgdp.csv'

# Load and preprocess the data
pce_df = load_and_preprocess_gdp_data(file_path)

# create hierarchy for GDP data
pce_df = create_structured_description(pce_df)

# create short description for GDP data
pce_df = create_short_description(pce_df)

#extract only PCE data
pce_df = pce_df[pce_df['short_description'].str.contains('PCE')]

# Transform the date formats
pce_df = transform_date_formats(pce_df)

#save the data
pce_df.to_csv('./results/bea/bea_pce_original.csv', index=False)

In [ ]:
pce_df.head()

<div style="color:#00BFFF">

##### Visualy inspect the BEA data


In [ ]:

def plot_time_series_with_iqr_and_extended_range_subplot(df, ax, column):
    # Use the index as it is already in datetime format
    datetime_index = df.index

    # Calculate statistics
    median = df[column].median()
    std = df[column].std()
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = median - 2.698 * std
    upper_whisker = median + 2.698 * std
    
    # Plot the time series line graph
    ax.plot(datetime_index, df[column], marker='o', markersize=3, color='black', linewidth=1, label='PCE')

    # Shade the IQR
    ax.fill_between(datetime_index, Q1, Q3, color='grey', alpha=0.4, label='IQR')
    
    # Shade the extended range
    ax.fill_between(datetime_index, lower_whisker, upper_whisker, color='lightgrey', alpha=0.3, label='Extended Range')
    
    # Mark potential outliers
    outliers = df[column][(df[column] < lower_whisker) | (df[column] > upper_whisker)]
    ax.scatter(outliers.index, outliers, color='red', zorder=5, label='Outliers')

    # Add median line
    ax.axhline(median, color='darkgreen', linestyle='--', linewidth=1.0, label='Median')
    
    # Add upper and lower whiskers lines
    ax.axhline(upper_whisker, color='grey', linestyle='--', linewidth=1, label='Upper Whisker')
    ax.axhline(lower_whisker, color='grey', linestyle='--', linewidth=1, label='Lower Whisker')

    # Add labels and legend
    ax.set_xlabel('Time')
    ax.set_ylabel(column)
    ax.set_title(f'{column}')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(False)
    
def plot_stacked_area_chart(data, ax):
    # Plotting the stacked area chart for specified columns
    ax.stackplot(data.index, data['PCE_Goods_Durable_goods'], data['PCE_Goods_Nondurable_goods'], data['PCE_Services'],
                 labels=['PCE_Goods_Durable_goods','PCE_Goods_Nondurable_goods', 'PCE_Services'], 
                 alpha=0.5)

    # Add labels and legend
    ax.set_xlabel('Time')
    ax.set_ylabel('Expenditure')
    ax.set_title('Stacked Area Chart of PCE Components')
    ax.legend(loc='upper left')
    ax.grid(False)



The rate of change is typically calculated as 
(
Current Value
−
Previous Value
Previous Value
)
×
100
%
( 
Previous Value
Current Value−Previous Value
​	
 )×100%, which can be easily computed using the pct_change() function in pandas, and then multiplying by 100 to convert it to a percentage.

In [ ]:
def analyze_and_plot(pce_df, columns_for_correlation):
    """
    Function to analyze and plot data from the given DataFrame.
    """

    # Calculate the rate of change for each column
    pce_real_growth = pce_df.pct_change().dropna() * 100

    # Selecting the relevant columns for correlation
    correlation_data = pce_df[columns_for_correlation]

    # Plotting
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))
    fig.tight_layout(pad=4.0)

    plot_stacked_area_chart(pce_df, axs[0])
    plot_time_series_with_iqr_and_extended_range_subplot(pce_real_growth, axs[1], 'PCE')

    axs[1].set_title('Rate of Change for PCE')
    plt.show()

    # Display the correlation matrix
    correlation_matrix = correlation_data.corr()
    return correlation_matrix

# Example usage
columns_for_correlation = ['PCE', 'PCE_Goods', 'PCE_Goods_Durable_goods','PCE_Goods_Nondurable_goods', 'PCE_Services']
result = analyze_and_plot(pce_df, columns_for_correlation)

print(f'Correlation matrix for PCE and its components:')
result


**Interpretation of Scatter Plots with Regression Lines:**

Scatter plots with regression lines for each component ('PCE_Goods_Durable_goods', 'PCE_Goods_Nondurable_goods', 'PCE_Services') against 'PCE' illustrate the linear relationships between these components and the total PCE. The regression lines provide a visual indicator of the direction, strength, and linearity of these relationships.

**Interpretation of the Correlation Matrix:**

The correlation matrix, particularly its visualization through the heatmap, shows the Pearson correlation coefficients between 'PCE' and its components. The coefficients near 1 indicate a very strong positive linear relationship.

<div style="color:#00BFFF">

##### Loading the FRED data

The `load_fredmd_data` function, below, performs the following actions, once for the FRED-MD dataset and once for the FRED-QD dataset:

1. Based on the `vintage` argument, it downloads a particular vintage of these datasets from the base URL https://files.stlouisfed.org/files/htdocs/fred-md into the `orig_m` variable.
2. Extracts the column describing which transformation to apply, shortname and description mappings
3. Extracts the observation date (from the "sasdate" column) and uses it as the index of the dataset.

In [ ]:
def load_fredmd_data(vintage):
    """
    Loads and processes the FRED-MD data.
    """
    # Define the base URL for the FRED-MD dataset
    base_url = 'https://files.stlouisfed.org/files/htdocs/fred-md'

    # Load the dataset for the specified 'vintage', dropping rows that are entirely NA
    fred_orig = pd.read_csv(f'{base_url}/monthly/{vintage}.csv').dropna(how='all')

    # Extract transformation codes (second column onwards) from the first row
    transform_info = fred_orig.iloc[0, 1:]

    # Drop the first row (containing transformation info) from the dataset
    fred_orig = fred_orig.iloc[1:]

    # Convert 'sasdate' column to a PeriodIndex with monthly frequency for time-series analysis
    fred_orig.index = pd.PeriodIndex(fred_orig.sasdate.tolist(), freq='M')

    # Remove the 'sasdate' column as it's now set as the index
    fred_orig.drop('sasdate', axis=1, inplace=True)

    # Return the processed data and the transformation information
    return fred_orig, transform_info


# Load data for the current vintage and unpack into original data and transformation info
fred_orig, transform_info = load_fredmd_data("current")

# Save the original data
fred_orig.to_csv('./results/fred/fred_monthly_orig.csv', index=False)

fred_orig.head(2)

<div style="color:#00BFFF">

##### Mapping FRED indices to Economic Data groups for analysis per economic group

In this section, we import and organize the definitions of economic variables. 
These definitions are loaded from CSV files corresponding to the FRED-MD and FRED-QD databases. 
This process ensures that we have a clear and concise understanding of each economic variable in our dataset, which is essential for accurate analysis and interpretation of the data.

In [ ]:
# Function for Column Name Mapping
def map_column_names(data, defn_file):
    """
    Maps FRED-MD column names to their descriptions.
    """
    # Load the definitions file, ignoring encoding errors
    defn = pd.read_csv(defn_file, encoding_errors='ignore')

    # Set the 'fred' column as the index of the definitions DataFrame
    defn.index = defn.fred

    # Filter the definitions to include only those variables present in the data columns
    defn = defn.loc[data.columns.intersection(defn.fred), :]

    # Create a dictionary mapping FRED-MD variable names to their descriptions
    map_dict = defn['description'].to_dict()

    # Replace the names of columns in the dataset with the descriptions from the map
    return data[map_dict.keys()].rename(columns=map_dict),defn

# Map column names for fred_original 
column_defn_file = './data/FRED/FRED_Definitions_Mapping/fredmd_definitions.csv'
fred_orig,defn = map_column_names(fred_orig, column_defn_file)

fred_orig.head(2)

Below, we get the groups for each series from the definition files above, and then show how many of the series that we'll be using fall into each of the groups.

We'll also re-order the series by group, to make it easier to interpret the results.

In [ ]:
# Get the mapping of variable id to group name, for monthly variables
groups = defn[['description', 'group']].copy()

# save the groups
defn.to_csv('./results/fred/fred_indicator_mappings.csv', index=False)

# Display the number of variables in each group
(groups.groupby('group', sort=False)
       .count()
       .rename({'description': '# series in group'}, axis=1))

<div style="color:#00BFFF">

### Filtering the data on date range and indicators

In [ ]:
# Filter the data to include only observations from the year 1980 onwards
fred_orig = fred_orig[fred_orig.index.year >= 1960]

# Filter the data to include only observations from the year 1980 onwards
pce_df = pce_df[pce_df.index.map(lambda x: int(x[:4]) >= 1960)]


Housing	Housing Starts, Midwest	Natural log: ln(x)	Thousands of Units
Housing	Housing Starts, Northeast	Natural log: ln(x)	Thousands of Units
Housing	Housing Starts, South	Natural log: ln(x)	Thousands of Units
Housing	Housing Starts, West	Natural log: ln(x)	Thousands of Units
Housing	Housing Starts: Total New Privately Owned	Natural log: ln(x)	Thousands of Units
Housing	New Private Housing Permits (SAAR)	Natural log: ln(x)	Thousands, Seasonally Adjusted Annual Rate
Housing	New Private Housing Permits, Midwest (SAAR)	Natural log: ln(x)	Thousands, Seasonally Adjusted Annual Rate
Housing	New Private Housing Permits, Northeast (SAAR)	Natural log: ln(x)	Thousands, Seasonally Adjusted Annual Rate
Housing	New Private Housing Permits, South (SAAR)	Natural log: ln(x)	Thousands, Seasonally Adjusted Annual Rate
Housing	New Private Housing Permits, West (SAAR)	Natural log: ln(x)	Thousands, Seasonally Adjusted Annual Rate

Seasonal Adjustment: For non-seasonally adjusted data, you should consider applying seasonal adjustment first to remove seasonal effects and better isolate the underlying trends. However, as your data appears to be seasonally adjusted (as indicated by SAAR - Seasonally Adjusted Annual Rate), this step may already be done.

In [ ]:


# drop the individual regional columns and just keep total 'Housing Starts: Total New Privately Owned' and 'New Private Housing Permits (SAAR)' column
columns_to_drop = [
    'Housing Starts, Midwest',
    'Housing Starts, Northeast',
    'Housing Starts, South',
    'Housing Starts, West',
    'New Private Housing Permits, Midwest (SAAR)',
    'New Private Housing Permits, Northeast (SAAR)',
    'New Private Housing Permits, South (SAAR)',
    'New Private Housing Permits, West (SAAR)',
    'Ratio of Help Wanted/No. Unemployed' #drop columns described by literature as not useful
    ]
fred_orig.drop(columns=columns_to_drop, inplace=True)


# New Orders for Consumer Goods

# Consumer Sentiment Index only last 30 years
# fred_orig.head()


<div style="color:#00BFFF">

### Frequency Alignment and Intergration

<div style="color:#00BFFF">

##### Frequency Alignment: 
</div>
- Transform the monthly economic indices from FRED to a quarterly format to align with the BEA’s quarterly GDP data. Calculate the sum or average (as appropriate) of monthly values within each quarter. 


In [ ]:
# Function to transform monthly date to quarterly date in 'YYYYQX' format
def transform_to_quarterly(date_str):
    year, month = date_str.split('-')
    quarter = (int(month) - 1) // 3 + 1
    return f"{year}Q{quarter}"

# Convert index to string to apply string methods
fred_orig.index = fred_orig.index.astype(str)

# Selecting only the last month of each quarter from the monthly dataset
# The last month of each quarter are March (03), June (06), September (09), December (12)
fred_orig_filtered = fred_orig[fred_orig.index.str.endswith(('03', '06', '09', '12'))]

# Transform the index to the quarterly format and create a new 'Quarter' column
fred_orig_filtered['Quarter'] = fred_orig_filtered.index.map(transform_to_quarterly)

# Set the new 'Quarter' column as the index
fred_orig_filtered.set_index('Quarter', inplace=True)

# Checking the date range of the monthly dataset
monthly_date_range = fred_orig_filtered.index.min(), pce_df.index.max()

# Checking the date range of the PCE dataset
pce_date_range = pce_df.index.min(), pce_df.index.max()

monthly_date_range, pce_date_range



<div style="color:#00BFFF">

##### Quarterly Data Integration

</div>
- We will merge quarterly BEA PCE rate of change data framework with the FRED transformed quarter-over-quarter rate of change into a unified framework using pandas, ensuring seamless integration and compatibility. 
- This step is vital for consolidating different economic indicators into a single, comprehensive analysis.

In [ ]:
# drop columns PCE_Goods	PCE_Goods_Durable_goods	PCE_Goods_Nondurable_goods	PCE_Services
pce_df.drop(['PCE_Goods', 'PCE_Goods_Durable_goods', 'PCE_Goods_Nondurable_goods', 'PCE_Services'], axis=1, inplace=True)

# Merging the datasets on the 'Quarter' column
joined_dataset = pd.merge(pce_df, fred_orig_filtered, left_index=True, right_index=True, how='left')

# Display the first few rows of the merged dataset
joined_dataset.head()


In [ ]:
joined_dataset.to_csv("./results/merged_data/joined_dataset.csv")

<div style="color:#00BFFF">

### Inspect and Handle Outliers


- Outliers are defined as observations that deviate significantly from the series mean, specifically those that are more than 10 times the interquartile range (IQR) away from the mean.
- To carry out this outlier removal, we've created a function named `remove_outliers`. This function identifies extreme values and replaces them with NaN (missing values). It also keeps track of the year in which each extreme value was removed.

In [ ]:
#inspect meausure types for each indicator to understand how to treat outliers

#map columns to measure type and remove outlier according to measure type
measuremnet_info = pd.read_csv('./data/fredmd_information.csv')

# Create a dictionary mapping FRED-MD variable names to their descriptions
measure_type_dict = measuremnet_info.set_index('description')['measure'].to_dict()

# Add "PCE": 'billions of dollars' to the dictionary
measure_type_dict["PCE"] = 'billions of dollars'

#display the measure types in a dataframe
pd.DataFrame.from_dict(measure_type_dict, orient='index', columns=['measure']).head()

Handles outliers in a DataFrame column based on the measure type and reports the removed outliers.

Args:
df (DataFrame): The DataFrame containing the data.
column (str): The name of the column to process.
measure (str): The type of measure for the column.

Returns:
DataFrame, DataFrame: The DataFrame with outliers handled, and DataFrame of removed outliers.

In [ ]:
def handle_outliers(df, column, measure):
    
    outlier_values = pd.DataFrame()

    if measure in ['avg dollars per hour', 'billions of 1982-84 dollars)', 'billions of dollars', 
                   'millions of dollars', 'exchange rate', 'index = 100', 'billions of chained 2012 dollars',
                   'billions of 2012 dollars)', 'millions of 2012 dollars, deflated by core pce', 
                   'billions of dollars, adjusted for inflation and excluding government transfer payments.']:
        # Z-score method for dollar values and indexes
        z = np.abs((df[column] - df[column].mean()) / df[column].std())
        threshold = 3  # Typically, a threshold of 3 is used
        mask = z > threshold
        outlier_values = df.loc[mask, column].reset_index()
        df.loc[mask, column] = np.nan

    elif measure in ['avg hours', 'avg no of weeks', 'avg dollars per hour', 'thousands of units']:
        # IQR method for averages and units
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mask = ~df[column].between(lower_bound, upper_bound)
        outlier_values = df.loc[mask, column].reset_index()
        df.loc[mask, column] = np.nan

    elif measure in ['percent', 'ratio' ]: #'thousands of persons'
        # Percentile-based method for percentages and ratios
        lower_bound = df[column].quantile(0.03)
        upper_bound = df[column].quantile(0.97)
        mask = ~df[column].between(lower_bound, upper_bound)
        outlier_values = df.loc[mask, column].reset_index()
        df.loc[mask, column] = np.nan

    return df, outlier_values

In [ ]:

# Initialize a dictionary to store outlier data for each column
all_outliers = {}
columns_with_outliers = {}

# Loop through each column in fred_orig and apply the outlier handling function
for column in fred_orig.columns:
    # Get the measure type for the column
    measure_type = measure_type_dict.get(column)
    if measure_type:
        # Apply the outlier handling function
        fred_orig, outliers = handle_outliers(fred_orig, column, measure_type)
        
        # Store the outliers if any
        if not outliers.empty:
            all_outliers[column] = outliers
            # columns_with_outliers.append(column)
            #add to dictionary columns_with_outliers
            columns_with_outliers[column] = outliers.count()[1]

# convert dictionary to dataframe and sort by number of outliers
columns_with_outliers = pd.DataFrame.from_dict(columns_with_outliers, orient='index', columns=['outliers']).sort_values(by='outliers', ascending=False)
columns_with_outliers

<div style="color:#00BFFF">

### Log Transformation on joined dataset for comparability

**Rationale:** 

Logarithmic transformation is used to stabilize the variance in data that exhibits exponential growth or large fluctuations. This is especially crucial for datasets like FRED's, where certain indicators can show significant variability over time. Given the information from the FRED database and their suggested transformation types, it seems reasonable to align with their expertise and apply these transformations to your dataset. This approach will save time and ensure that the data is treated consistently with established economic analysis practices.

**Transformation Types (as per FRED):**

1. **No Transformation (1)**: The data is used as is, without any modification.
   
2. **First Difference (∆x_t) (2)**: The change from one period to the next, useful for highlighting trends.
   
3. **Second Difference (∆^2x_t) (3)**: The change in the first difference, often used to capture acceleration or deceleration in a series.
   
4. **Natural Log (log(x_t)) (4)**: Useful for stabilizing variance and making exponential growth trends linear.
   
5. **First Difference of Log (∆ log(x_t)) (5)**: Commonly used to convert data into a stationary series, representing percentage change.
   
6. **Second Difference of Log (∆^2 log(x_t)) (6)**: The change in the first difference of the log, similar to the second difference but for logged data.
   
7. **Percentage Change from Prior Period (∆(x_t/x_t_−_1 − 1.0)) (7)**: This calculates the percentage change from the previous period, emphasizing relative changes.

**Approach:**

- **Apply Transformations:** Apply FRED Transformations and use the transformation codes provided in your `fred_indicator_mappings` dataset to transform the corresponding series in your `pce_joined_dataset`.
- This approach should streamline our analysis process and align with the methodology with FRED's established practices. 
- Additionally, it ensures that the data is treated in a manner that is suitable for economic analysis.
-  **FRED Logarithmic Key Mapping:** We will map the transformation codes in the FREDmd_defn dataset to our dataset's indicators and then perform the necessary transformations.

In [ ]:
#  transformation function to handle the time column and a special case for PCE
def modified_log_transform(column, time_column, transformation_code, column_name):
    """
    Applies the specified transformation to a Pandas Series, considering the time column and special cases.
    """
    # Special instruction for the PCE column
    if column_name == "PCE":
        transformation_code = 5  #6 # according to FREDs guidelines
    # elif column_name == "IP: Nondurable Consumer Goods":
    #     transformation_code = 3 #previous mapping to was still non-stationary
    # elif (column_name == "Housing Starts: Total New Privately Owned") or "New Private Housing Permits (SAAR)":
    #     transformation_code = 2 #previous mapping to 4 was still non-stationary

    # Check if the data is quarterly based on the time column
    mult = 4 if any(time_column.str.endswith(('Q1', 'Q2', 'Q3', 'Q4'))) else 1

    if transformation_code == 1:
        # No transformation -> Mathematical Equation: x(t)
        # It leaves the data in its original form, without any alteration.
        return column
    
    elif transformation_code == 2:
        # First Difference -> Mathematical Equation: x(t) - x(t-1)
        # It measures the absolute change from one period to the next, helping to detrend the data.
        return column.diff()
    
    elif transformation_code == 3:
        # Second Difference -> Mathematical Equation: (x(t) - x(t-1)) - (x(t-1) - x(t-2))
        # It measures the change in the first difference, capturing the acceleration or deceleration in the data's movement.
        return column.diff().diff()
    
    elif transformation_code == 4:
        # Log Transformation -> Mathematical Equation: ln(x(t))
        # It stabilizes the variance across the data series and can help make a skewed distribution more normal.
        return np.log(column)
    
    elif transformation_code == 5:
        # Log First Difference -> Mathematical Equation: 100 * (ln(x(t)) - ln(x(t-1)))
        # It measures the growth rate from one period to the next and multiplies by 100 for percentage change.
        # The 'mult' variable allows for scaling the growth rate if necessary.
        return np.log(column).diff() * 100 * mult
    
    elif transformation_code == 6:
        # Log Second Difference -> Mathematical Equation: 100 * ((ln(x(t)) - ln(x(t-1))) - (ln(x(t-1)) - ln(x(t-2))))
        # It measures the change in the growth rate (change in log first difference), capturing the momentum of change.
        # The 'mult' variable allows for scaling the change in growth rate if necessary.
        return np.log(column).diff().diff() * 100 * mult
    
    elif transformation_code == 7:
        # Exact Percent Change -> Mathematical Equation: 100 * ((x(t)/x(t-1))^mult - 1)
        # It measures the percentage change from one period to the next, with an option to compound the change using 'mult'.
        return ((column / column.shift(1))**mult - 1.0) * 100
    
    else:
        raise ValueError("Invalid transformation code")


# Create a mapping of columns to transformation codes
transformation_mapping = defn.set_index('description')['tcode'].to_dict()

# Extracting the time column
time_column = joined_dataset.index

# Applying the transformations to the dataframe
transformed_dataset = joined_dataset.copy()

for column in transformed_dataset.columns:
    # Check if the column is in the mapping, else apply special instruction for PCE
    tcode = transformation_mapping.get(column, None)
    transformed_dataset[column] = modified_log_transform(transformed_dataset[column], time_column, tcode, column)

# Drop the first 5 rows containing NaN values resulting from the transformation
transformed_dataset = transformed_dataset.iloc[5:]

# Displaying the first few rows of the transformed dataset
joined_dataset = transformed_dataset
joined_dataset.head(8)


<div style="color:#00BFFF">

### Inspecting and Handling Missing Values

In [ ]:
from sklearn.impute import SimpleImputer

# Check for NaN values in the dataset
nan_summary = joined_dataset.isna().sum()

# Sort the NaN summary to identify columns with the most missing values
nan_summary_sorted = nan_summary.sort_values(ascending=False)

# Display columns with NaNs and their count
columns_with_nans = nan_summary_sorted[nan_summary_sorted > 0]
columns_with_nans


In [ ]:
#save New Orders for Consumer Goods and Consumer Sentiment Index in seperate dataframe and drop from joined_dataset for later seperate correlation testing
new_orders_for_consumer_goods = joined_dataset['New Orders for Consumer Goods']
consumer_sentiment_index = joined_dataset['Consumer Sentiment Index']

#drop columms from joined_dataset
joined_dataset.drop(['New Orders for Consumer Goods', 'Consumer Sentiment Index'], axis=1, inplace=True)

#save the two dataframes
new_orders_for_consumer_goods.to_csv('./results/csi_and_new_orders/new_orders_for_consumer_goods.csv')
consumer_sentiment_index.to_csv('./results/csi_and_new_orders/consumer_sentiment_index.csv')

In [ ]:
# from sklearn.impute import SimpleImputer

# def handle_and_display_missing_values(df):

#     # Forward fill, then backward fill
#     df.bfill(inplace=True)
#     df.ffill(inplace=True)
    
#     # Linear interpolation
#     #df.interpolate(method='linear', inplace=True)

#     # Recheck for NaN values after interpolation
#     nan_summary_after_interpolation = df.isna().sum()
#     print("\nNaN values after interpolation and forward/backward fill:")
#     print(nan_summary_after_interpolation[nan_summary_after_interpolation > 0])

#     # Median imputation for any remaining missing values
#     if nan_summary_after_interpolation.any():
#         imputer = SimpleImputer(strategy='median')
#         df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

#     # Final check for NaN values
#     nan_summary_after_all_imputations = df.isna().sum()
#     print("\nFinal NaN values summary:")
#     print(nan_summary_after_all_imputations[nan_summary_after_all_imputations > 0])

#     return df

# # Assuming 'joined_dataset' is your DataFrame
# handled_dataset = handle_and_display_missing_values(joined_dataset)


<div style="color:#00BFFF">

### Stationary Assesment for Joined Dataset


**Stationarity Assessment**: Using tests like the Augmented Dickey-Fuller ensures that our time series data is suitable for modelling and forecasting, as many statistical models require stationarity for valid results.

**Addressing Non-Stationarity**: Techniques such as differencing or transformation will be applied to achieve stationarity, which is crucial for the accuracy and reliability of our predictive models and correlation analysis. 

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Function to perform Augmented Dickey-Fuller test for stationarity
def adf_test(series, signif=0.05):

    # Replace infinities with NaNs and then fill them
    series.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Now run the ADF test
    result = adfuller(series, autolag='AIC')
    p_value = result[1]
    return p_value < signif

#get join_dataset column names in a list and place in key_indicators_for_var to contain valid column names
key_indicators_for_var = joined_dataset.columns.tolist()

# Checking stationarity
stationarity_results = {}
for indicator in key_indicators_for_var:
    if indicator in joined_dataset.columns:
        stationarity_results[indicator] = adf_test(joined_dataset[indicator].dropna())
    else:
        print(f"Indicator {indicator} not found in dataset")

# Print non-stationary indicators
print("Non-stationary Indicators:")
for key, value in stationarity_results.items():
    if not value:
        print(key)

In [ ]:
#treat non stationary indicators

#to be conducted

<div style="color:#00BFFF">

### Create Additional dataset with Differencing (through Rate of Change Q-o-Q) 

**Objective:** To create secondary dataset to compare different economic indicators on a common scale.

**Method:** We normalize the rate of change of various indicators from one period to the next. This standardization facilitates more meaningful analysis across diverse data points, as it accounts for differences in magnitude and unit measurements.

Using the rate of change as a standardization method for these variables can be quite effective, especially in the context of economic data and nowcasting models. This approach has several advantages:

**Advantages of Using Rate of Change**

1. **Comparability**: It allows for a more meaningful comparison across different indicators, which may have different scales and units.

2. **Trend Analysis**: Rate of change emphasizes trends and growth rates, which are often more informative for economic analysis than absolute levels.

3. **Stationarity**: Economic time series data often need to be stationary for effective modeling. Rates of change can help in achieving stationarity, a common requirement for many time series models.

4. **Handling Non-Linearity**: Log transformations followed by calculating rates of change can linearize exponential growth patterns, making the data more suitable for linear modeling techniques.

5. **Economic Relevance**: Rates of change are inherently more meaningful in economic analysis. For instance, policymakers and analysts are often more interested in the growth rate of GDP rather than its absolute level.

**Considerations**

However, there are a few considerations to keep in mind:

- **Loss of Level Information**: By focusing on rates of change, you lose information about the absolute levels, which can sometimes be relevant.
- **Volatility**: Rates of change can sometimes amplify volatility, especially in series with small absolute numbers.
- **Interpretability**: Ensure that the transformed data remains interpretable and aligned with economic intuition.

In [ ]:
joined_dataset_rate_of_change = joined_dataset.pct_change()

joined_dataset_rate_of_change = joined_dataset_rate_of_change.replace([np.inf, -np.inf], np.nan)

# joined_dataset_rate_of_change = handle_and_display_missing_values(joined_dataset_rate_of_change)

joined_dataset_rate_of_change.head(3)

<div style="color:#00BFFF">

##### Standardizing rate of change dataframe (Z-Score Normalization) 

By rescaling the data to have a mean of zero and a standard deviation of one, we facilitate multivariate analyses and make our variables comparable in terms of variation from their mean growth. This is crucial for regression models and techniques like PCA where scale impacts the results significantly.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data and transform it and applying this only to the numeric columns
scaled_data = scaler.fit_transform(joined_dataset_rate_of_change.select_dtypes(include=['float64', 'int64']))

# Create a new DataFrame with the scaled data and assumes that the index contains non-numeric columns like dates
joined_dataset_rate_of_change = pd.DataFrame(scaled_data, index=joined_dataset_rate_of_change.index, columns=joined_dataset_rate_of_change.select_dtypes(include=['float64', 'int64']).columns)

joined_dataset_rate_of_change.head()


<div style="color:#00BFFF">

### Initial Correlation Analysis on the Transformed Data and Rate of Change Data

**Why**: 

Given our scenario where we are analyzing a large number of indicators (123) across a lengthy period (1960 to 2023) to find those that best correlate with private consumption expenditure, and we want to `retain NaN` values due to their economic significance, `Spearman's rank correlation` with pairwise deletion seems to be the most appropriate. 

It respects the economic significance of NaN values while providing a robust correlation measure. However, we need to ensure to perform a sensitivity analysis to understand the impact of missing values on your results. Also, consider the potential for non-random missing data and its implications.

**What**: 
`Spearman's rank correlation` is non-parametric and does not assume a linear relationship between variables, which can be more appropriate for economic data. It also handles NaN values by default in many implementations, like in Python's Pandas library, where it ignores pairs where either value is NaN. 

**Benefits**: 

Accounts for monotonic relationships and is less sensitive to outliers (which is relevant given that you have replaced some extreme values with NaN).

**Spearman's rank correlation for log transformed data**

In [ ]:
# Calculate the Spearman's rank correlation with the private consumption expenditure, handling NaNs with pairwise deletion.
correlation_matrix = joined_dataset.corr(method='spearman')

#target_correlations will have the Spearman's rank correlation coefficients
target_correlations = correlation_matrix['PCE'].sort_values(ascending=False)

# Display the correlations related to 'PCE'
print(target_correlations.head(10))
print('\n')
print(target_correlations.tail(10).sort_values(ascending=True))

**Spearman's rank correlation for rate of change data**

In [ ]:
# Calculate the Spearman's rank correlation with the private consumption expenditure, handling NaNs with pairwise deletion.
correlation_matrix_rate_of_change = joined_dataset_rate_of_change.corr(method='spearman')

#target_correlations will have the Spearman's rank correlation coefficients
target_correlations_rate_of_change = correlation_matrix['PCE'].sort_values(ascending=False)

# Display the correlations related to 'PCE'
print(target_correlations_rate_of_change.head(10))
print('\n')
print(target_correlations_rate_of_change.tail(10).sort_values(ascending=True))

<div style="color:#00BFFF">

### Save the final cleaned and preprocessed dataframe

In [ ]:
# save the data
joined_dataset.to_csv("./results/merged_data/joined_dataset_transformed.csv",index=True)
joined_dataset_rate_of_change.to_csv("./results/merged_data/joined_dataset_rate_of_change.csv",index=True)